# Discrete SISO Identification using Unregularized Least Squares

In [ ]:
%pylab inline
import numpy as np
import pylab
try:
    import seaborn as sns  # optional; prettier graphs
except ImportError:
    pass

from scipy.signal import lfilter
from nengo.solvers import Lstsq
from nengo.utils.numpy import rmse

import nengolib

Create an arbitrary system (here, an Alpha synapse) and discretize it. 

In [ ]:
sys = nengolib.Alpha(0.005)
dsys = nengolib.signal.cont2discrete(sys, dt=0.001)
n_num, n_den = dsys.order_num, dsys.order_den
num, den = dsys.num, dsys.den

Filter some white-noise input $u[k]$ with this system to get an output $y[k]$. Our goal is to estimate the discrete SISO transfer function with order ($n_{num}$ / $n_{den}$) given only access to $(u, y)$.

In [ ]:
u = np.random.normal(size=500)
y = lfilter(dsys.num, dsys.den, u)

pylab.figure()
pylab.plot(u)
pylab.plot(y)
pylab.show()

This can be formulated as a least-squares problem $A\theta = Y$ using the [linear difference equation](https://en.wikipedia.org/wiki/Z-transform#Linear_constant-coefficient_difference_equation) form of the discrete transfer function (see [1] for extra details). Here, $A$ is given by appropriately time-shifting the $(u, y)$ data, $Y$ is the desired result from shifting $y$, and $\theta$ are the combined transfer function parameters that we are to estimate.

In [ ]:
n = len(u) - n_den

A = np.empty((n, n_num+1+n_den))
for i in range(n_num+1):
    A[:, i] = u[n_den-i:n_den-i+n]
for i in range(n_den):
    A[:, n_num+1+i] = -y[n_den-1-i:n_den-1-i+n]
    
Y = y[n_den:n_den+n]

First, we verify that the true transfer function gives the ideal $\theta$ such that $A\theta = Y$. 

In [ ]:
assert np.allclose(den[n_den], 1)
theta = np.append(num, np.asarray(den)[1:])  # discard leading 1

def check_tf(theta):
    pylab.figure()
    pylab.plot(np.dot(A, theta))
    pylab.plot(Y)
    pylab.show()
    return rmse(np.dot(A, theta), Y)
    
print theta, check_tf(theta)

Now, we use a least-squares solver to get an estimate $\hat{\theta}$, and check that $A\hat{\theta} \approx Y$. In fact, we get back the same transfer function within machine-epsilon, due to the absence of any noise in the problem.

In [ ]:
theta_est, _ = Lstsq()(A, Y)
print theta_est, check_tf(theta_est)
assert np.allclose(theta, theta_est)

Finally, this can be converted back into a transfer function, and used to simulate the same $y$.

In [ ]:
num_est = theta_est[:n_num+1]
den_est = np.append([1], theta_est[n_num+1:])

y_est = lfilter(num_est, den_est, u)

pylab.figure()
pylab.plot(y)
pylab.plot(y_est)
pylab.show()

assert np.allclose(y, y_est)

### References 

[1] Ljung, Lennart, and Tianshi Chen. "What can regularization offer for estimation of dynamical systems?." *11th IFAC International Workshop on Adaptation and Learning in Control and Signal Processing (ALCOSP13), 3-5 July 2013, Caen, France*. IFAC, 2013.